# Combining Pre-Built & Custom AI Services

In this notebook, you will integrate with the Computer Vision API and the Text Analytics API to augment the claims processing capabilities. In the end, you will integrate the API calls to the summarizer and classifier services that your deployed and produce a finished claim report that shows all of the processing applied to the claim text and claim image.

### Setup helper functions

Run the cell below to enable helper functions to save locally the outputs as pickle files from the various cognitive services 

In [ ]:
import pickle
import os

output_location = './cs_outputs'
os.makedirs(output_location, exist_ok=True)

def save_output(output, name):
    file_name = os.path.join(output_location, name) + '.pkl'
    with open(file_name, 'wb') as f:
        pickle.dump(output, f, pickle.HIGHEST_PROTOCOL)
        
def get_output(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)

## Task 1 - Caption & Tag with the Computer Vision API

In the cell bellow, provided the key to your Computer Vision API and run the cell.

In [ ]:
subscription_key = '' #"<your_computer_vision_api_key>"
assert subscription_key

Construct the endpoint to the Computer Vision API by running the following cell. Notice the last path segment is analyze, which indicates you will use the analyze feature.

Be sure to update the value in vision_endpoint below so it matches the Endpoint value you copied from the Azure Portal for your instance of the Computer Vision service. Be sure your value ends in a slash (/).

In [ ]:
vision_endpoint = '' #"<your_computer_vision_api_endpoint>"
vision_base_url = vision_endpoint + "vision/v3.0/"
vision_analyze_url = vision_base_url + "analyze"

The following cell contains a list of sample images found after performing a simple web search. Feel free to substitute in URLs to the image of your choice.

In [ ]:
fender_bender = "https://www.washingtonpost.com/blogs/innovations/files/2015/02/Stolen_Car_Crash-00aef.jpg"
damaged_house = "https://c2.staticflickr.com/8/7342/10983313185_0589b74946_z.jpg"
police_car = "https://localtvwnep.files.wordpress.com/2015/11/fender-bender.jpeg"
car_with_text = "https://static.buildasign.com/cmsimages/bas-vinyl-lettering-splash-01.png"
car_tow = 'https://i.ytimg.com/vi/wmxJ2FrzTWo/maxresdefault.jpg'

From the list of images above, select one and assign it to image_url for further processing:

In [ ]:
image_url = car_tow

Run the following cell to preview the image you have selected.

In [ ]:
from IPython.display import Image, display
display(Image(image_url))

The following cell builds the HTTP request to make against the Computer Vision API.

Run the following cell to retrieve the caption and tags:

In [ ]:
import requests
headers  = {'Ocp-Apim-Subscription-Key': subscription_key }
params   = {'visualFeatures': 'Categories,Description,Tags,Color'}
data     = {'url': image_url}
response = requests.post(vision_analyze_url, headers=headers, params=params, json=data)
response.raise_for_status()
analysis = response.json()
# Save the ouput locally
save_output(analysis, 'vision_results')
analysis

As you can see in the above output, the result is a nested document structure. Run the following cells to pull out the caption and top 3 tag results:

In [ ]:
caption = analysis["description"]["captions"][0]["text"].capitalize()
caption

In [ ]:
topTags = analysis["description"]["tags"][0:3]
topTags

## Task 2 - Performing OCR

In order to perform OCR with the Computer Vision service, you need to target the OCR endpoint.

Run the following cell to construct the right URL:

In [ ]:
vision_ocr_url = vision_base_url + "ocr"

Next, invoke the OCR endpoint with the following code and examine the result:

In [ ]:
headers  = {'Ocp-Apim-Subscription-Key': subscription_key }
params   = {}
data     = {'url': image_url}
response = requests.post(vision_ocr_url, headers=headers, params=params, json=data)
response.raise_for_status()
ocr_analysis = response.json()
# Save the ouput locally
save_output(ocr_analysis, 'ocr_results')
ocr_analysis

We have provided the following code for you to extract the text as a flat array from the results.

Run the following cell to extract the text items from the results document:

In [ ]:
import itertools
flatten = lambda x: list(itertools.chain.from_iterable(x))
words_list = [[ [w['text'] for w in line['words']]  for line in d['lines']] for d in ocr_analysis['regions']]
words_list = flatten(flatten(words_list))
print(list(words_list))

## Task 3 - Performing Sentiment Analysis

Sentiment Analysis is performed using the Text Analytics API.

Update the following cell with the key to your instance of the Text Analytics API and run the cell:

In [ ]:
text_analytics_subscription_key = '' #"<your_text_analytics_key>"
assert text_analytics_subscription_key

Update the following cell with the correct Endpoint URL for your deployed instance of the Text Analytics API and run the cell. Be sure your value ends in a slash (/).

In [ ]:
#"<your_text_analytics_api_endpoint>"
text_analytics_base_url = ''
sentiment_api_url = text_analytics_base_url + "text/analytics/v3.0/sentiment"

The following cell has a set of example claims you can use to test the measurement sentiment.

Run the cell:

In [ ]:
neg_sent = """We are just devastated and emotionally drained. 
The roof was torn off of our car, and to make matters
worse my daughter's favorite teddy bear was impaled on the street lamp."""
pos_sent = """We are just happy the damaage was mininmal and that everyone is safe. 
We are thankful for your support."""
neutral_sent = """I crashed my car."""
long_claim = """
I was driving down El Camino and stopped at a red light.
It was about 3pm in the afternoon. The sun was bright and shining just behind the stoplight.
This made it hard to see the lights. There was a car on my left in the left turn lane.
A few moments later another car, a black sedan pulled up behind me. 
When the left turn light changed green, the black sedan hit me thinking 
that the light had changed for us, but I had not moved because the light 
was still red. After hitting my car, the black sedan backed up and then sped past me.
I did manage to catch its license plate. The license plate of the black sedan was ABC123. 
"""

From the above list of claims, select one and assign its variable to claim_text to be used in the call to the Text Analytics API.

In [ ]:
claim_text = long_claim

The API requires you to submit a document of the following form.

Run the cell to build the request document:

In [ ]:
documents = {'documents' : [
    {'id': '1', 'language': 'en', 'text': claim_text}
]}

Now invoke the Text Analytics API and observe the result.

In [ ]:
headers   = {"Ocp-Apim-Subscription-Key": text_analytics_subscription_key}
response  = requests.post(sentiment_api_url, headers=headers, json=documents)
sentiments = response.json()
# Save the ouput locally
save_output(sentiments, 'sentiment_results')
sentiments

Please note that the Text Analytics API provides breakdown of sentiments at the individual sentence levels. Run the following cell to view the overall sentiment of the document and sentiment breakdown by sentences.

In [ ]:
score_interpretation = sentiments['documents'][0]['sentiment']
print('Overall document sentiment:', score_interpretation)
print('')
print('Sentiment breakdown by sentences')
for item in sentiments['documents'][0]['sentences']:
    print('Sentence:', item['text'].strip(), 'Sentiment:', item['sentiment'])

## Task 4 - Save the Results in Blob Store

Save the JSON responses that came from the various cognitive services to a permanent store like the blob storage for future reference.

We have been saving the JSON outputs as pickle files temporarily in a local directory. Let’s review the saved files.

In [ ]:
!ls $output_location

**Create the `Workspace` from the saved config file**

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()

**Upload the files from the local directory to the default blob store**

In [ ]:
datastore = ws.get_default_datastore()
datastore.upload(output_location, 
                 target_path = 'cs_results', 
                 overwrite = True, 
                 show_progress = True)

Navigate to your resource group’s storage account, select **Containers**, and then select the default blob store **azureml-blobstore-xxx**. Next, select the folder **cs_results** to view the saved files.


![Image showing uploaded results from the cognitive services APIs.](./images/blob_files.png 'Blobs')

## Task 5 - Invoking the Azure ML Deployed Services

Run the following cell to define a method that will be used to invoke your classifier and summarizer methods deployed using Azure Machine Learning service to Azure Container Instances:

In [ ]:
def invoke_service(ml_service_key, ml_service_scoring_endpoint, ml_service_input):
    headers   = {"Authorization": "Bearer " + ml_service_key}
    response  = requests.post(ml_service_scoring_endpoint, headers=headers, json=ml_service_input)
    result = response.json()
    return result

Configure the classifier invocation with the key and endpoint as appropriate to your deployed instance:

> This is the scoring URI you copied from the notebook named `04 Deploy Classifier Web Service.ipynb`.

In [ ]:
classifier_service_key = "" #leave this value empty if the service does not have authentication enabled
#"<your_classifier_scoring_url>"
classifier_service_scoring_endpoint = ''
classifier_service_input = [claim_text]

Invoke the classifier and observe the result:

In [ ]:
classifier_result = invoke_service(classifier_service_key, 
                                   classifier_service_scoring_endpoint, classifier_service_input)
classifier_result

In [ ]:
# Interpret the classifier result
classification = 'Auto Insurance Claim' if classifier_result == 1 else 'Home Insurance Claim' 
classification

Similarly, configure the key and scoring endpoint as appropriate to your summarizer service:

> This is the scoring URI you copied from the notebook named `02 Deploy Summarizer Web Service.ipynb`.

In [ ]:
summarizer_service_key = "" #leave this value empty if the service does not have authentication enabled
#"<your_summarizer_service_url>"
summarizer_service_scoring_endpoint = ''
summarizer_service_input = claim_text

Invoke the summarizer service and observe the result:

In [ ]:
summarizer_result = invoke_service(summarizer_service_key, summarizer_service_scoring_endpoint, 
                                   summarizer_service_input)
formatted_result =  summarizer_result[0].replace("\\n", " ").strip() if len(summarizer_result) > 0 else "N/A"
formatted_result

## Task 6 - Summarizing the Results

In this final task, you pull together all of the pieces to display the results of your AI based processing.

Run the following cell and examine the result.

In [ ]:
from IPython.core.display import HTML

displayTemplate = """
<div><b>Claim Summary</b></div>
<div>Classification: {}</div>
<div>Caption: {}</div>
<div>Tags: {}</div>
<div>Text in Image: {}</div>
<div>Sentiment: {}</div>
<div><img src='{}' width='200px'></div>
<div>Summary: </div>
<div><pre>{} </pre></div>
<div>&nbsp;</div>
<div>Claim:</div>
<div>{}</div>

"""
displayTemplate = displayTemplate.format(classification, caption, ' '.join(topTags), ' '.join(words_list), 
                                         score_interpretation, image_url, formatted_result, 
                                         claim_text)
display(HTML(displayTemplate))